# Tutorial

In [1]:
import os
import scanpy as sc

In [2]:
import sys
sys.path.append("../..")
from idtrack import *
from idtrack._track_tests import *

In [3]:
logger_config()
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"  # or any other local directory
dm = DatabaseManager("homo_sapiens", 107, "gene", local_dir, 79)  # 70

In [4]:
tt: TrackTests = TrackTests(dm)

2022-08-13 15:14:56 INFO:graph: The graph is being read.


In [5]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

In [6]:
adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Meyer_2021": [f"{dset0_dir}/meyer_2021.h5ad"],
    "MeyerNikolic_unpubl": [f"{dset0_dir}/meyer_nikolic_unpubl.h5ad"],
    "Barbry_unpubl": [f"{dset0_dir}/barbry.h5ad"],
    "Regev_2021": [
        f"{dset0_dir}/delorey_cryo.h5ad", f"{dset0_dir}/delorey_fresh.h5ad",
        f"{dset0_dir}/delorey_nuclei.h5ad"
    ],
    "Thienpont_2018": [f"{dset1_dir}/Lambrechts/lambrechts.h5ad"],
    "Budinger_2020": [f"{dset0_dir}/bharat.h5ad"],
    "Banovich_Kropski_2020": [f"{dset0_dir}/haberman.h5ad"],
    "Sheppard_2020": [f"{dset0_dir}/tsukui.h5ad"],
    "Wunderink_2021": [
        f"{dset0_dir}/grant_cryo.h5ad", f"{dset0_dir}/grant_fresh.h5ad"
    ],
    "Lambrechts_2021": [
        f"{dset0_dir}/wouters.h5ad" #, f"{dset0_dir}/wouters_labs.h5ad"
    ],
    "Zhang_2021": [f"{dset1_dir}/Liao/covid_for_publish.h5ad"],
    "Duong_lungMAP_unpubl": [f"{dset0_dir}/duong.h5ad"],
    "Janssen_2020": [f"{dset0_dir}/mould.h5ad"],
    "Sun_2020": [
        f"{dset0_dir}/wang_sub_batch1.h5ad", f"{dset0_dir}/wang_sub_batch2.h5ad",
        f"{dset0_dir}/wang_sub_batch3.h5ad", f"{dset0_dir}/wang_sub_batch4.h5ad"],
    "Gomperts_2021": [
        f"{dset0_dir}/carraro_ucla.h5ad", f"{dset0_dir}/carraro_cff.h5ad",
        f"{dset0_dir}/carraro_csmc.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Schiller_2020": [f"{dset0_dir}/mayr.h5ad"],
    "Misharin_Budinger_2018": [f"{dset0_dir}/reyfman_disease.h5ad"],
    "Shalek_2018": [f"{dset0_dir}/ordovasmontanes.h5ad"],
    "Schiller_2021": [f"{dset0_dir}/schiller_discovair.h5ad"],
    "Peer_Massague_2020": [f"{dset0_dir}/laughney.h5ad"],
    "Lafyatis_2019": [f"{dset0_dir}/valenzi.h5ad"],
    "Tata_unpubl": [f"{dset0_dir}/tata_unpubl.h5ad"],
    "Xu_2020": [f"{dset0_dir}/guo.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"]
}

In [ ]:
from collections import Counter
result = dict()
for ddadaata in adata_dict:
    
    adata = sc.read(adata_dict[ddadaata][0])
    gene_list = list(adata.var.index)
    gene_list_filtered, gene_list_converted, gene_list_lost = tt.unfound_correct(list(adata.var.index))
    
    for amk in [gene_list, gene_list_filtered]:
        really_lost = []
        for i in amk:
            if i not in tt.graph.nodes:
                really_lost.append(i)
        print(ddadaata, len(amk), len(really_lost))
    print(tt.identify_source(gene_list_filtered)[0])
    
    a = dict()
    for ind, i in enumerate(set(gene_list_filtered)):
        if ind % 100 == 0 or ind > len(gene_list_filtered)-5:
            progress_bar(ind, len(gene_list_filtered)-1)
        conv = tt.convert(i, None, None, 'ensembl_gene', prioritize_to_one_filter=True)
        a[i] = conv
    result[ddadaata] = a
    print(list(Counter([len(a[i]) if a[i] is not None else 0 for i in a]).most_common())[:10])
    print()


Kaminski_2020 45947 0
Kaminski_2020 45947 0
(('HGNC Symbol', 95), 30503)
Progress: |████████████████████| 100.0% 
[(1, 44331), (2, 859), (0, 403), (4, 83), (6, 79), (7, 63), (3, 57), (5, 49), (16, 17), (8, 5)]

Meyer_2021 20922 0
Meyer_2021 20922 0
(('HGNC Symbol', 105), 20412)
Progress: |████████████████████| 100.0% 
[(1, 20298), (2, 388), (6, 49), (4, 40), (3, 40), (5, 38), (7, 36), (0, 27), (8, 5), (16, 1)]



2022-08-13 15:22:27 WARNING:track: Number of converted IDs with small modifications: 18
2022-08-13 15:22:27 WARNING:track: Number of IDs not found in the graph: 50


MeyerNikolic_unpubl 33582 68
MeyerNikolic_unpubl 33532 0
(('HGNC Symbol', 93), 23387)
Progress: |██████████----------| 52.49% 

In [15]:
rc37 = Dataset(dm.change_server(37), narrow_search=False).initialize_external_conversion()
#rc38 = Dataset(dm.change_server(38), narrow_search=False).initialize_external_conversion()

2022-08-12 22:26:23 INFO:dataset: Comparison data frame is being constructed for releases: [107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79].


In [16]:
d=np.unique(rc37['id_db'])

In [17]:
[i for i in d if "YX65C7" in i]

['XXyac-YX65C7_A.2', 'XXyac-YX65C7_A.3', 'XXyac-YX65C7_A.4']

In [18]:
"PRED58" in d

True

In [19]:
amk = [i for i in a if i not in d]

In [20]:
print(len([i for i in a if i in d]))
print(len(amk))

58
113


In [21]:
np.array([i for i in amk if i.split('.')[0] not in d])

array(['RP11-442N24--B.1', 'RP11-99J16--A.2', 'RP11-59D5--B.2',
       'RP11-445L13--B.3', 'RP11-544L8--B.4', 'XXyac-YX65C7-A.2',
       'XXyac-YX65C7-A.3', 'RP11-524D16--A.3', 'RP11-453F18--B.1',
       'Metazoa-SRP', 'Y-RNA', 'XX-DJ76P10--A.2', '5S-rRNA', 'Y-RNA.1',
       'Y-RNA.2', 'RP11-1157N2--B.2', 'RP1-213J1P--B.1',
       'RP1-213J1P--B.2', 'RP4-633O19--A.1', 'RP4-754E20--A.5',
       'CTA-280A3--B.2'], dtype='<U16')

In [22]:
# sonra da combinatorial olarak _ to - degisikligi yap..
 
# np.array([i for i in amk if i.split('.')[0] not in d])

In [23]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate([i for i in a if i in d]):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,

In [24]:
np.unique(rc37[rc37["id_db"] == 'FLJ27365']["name_db"])

array(['UniProtKB Gene Name', 'Uniprot_gn'], dtype=object)

In [26]:
ex=dm.create_external_all()

In [36]:
np.unique(ex[ex['assembly'] == 37]['name_db'])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene',
       'RFAM'], dtype=object)

array(['RFAM'], dtype=object)

In [57]:
jkl = dict()
askdnmkajsdn = rc37["name_db"]
for ind, i in enumerate(a):
    print(ind, end=",")
    jkl[i] = set(askdnmkajsdn[rc37["id_db"]==i])


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [59]:
Counter([j for i in jkl for j in jkl[i]])

Counter({'Clone-based (Vega) gene': 377,
         'Clone_based_vega_gene': 377,
         'Clone-based (Vega)': 377,
         'Clone-based (Ensembl) gene': 87,
         'Clone-based (Ensembl)': 87,
         'Clone_based_ensembl_gene': 87,
         'EntrezGene': 41,
         'Uniprot_gn': 42,
         'HGNC Symbol': 20,
         'NCBI gene (formerly Entrezgene)': 41,
         'WikiGene': 17,
         'UniProtKB Gene Name': 42,
         'RFAM': 2})

In [6]:
df=dm.create_external_all()

In [5]:
ExternalDatabases(dm).give_list_for_case(give_type="db")

['EntrezGene',
 'UniProtKB Gene Name',
 'NCBI gene (formerly Entrezgene)',
 'HGNC Symbol']

In [8]:
np.sum(df['assembly']==38)

202607

In [9]:
np.sum(df['assembly']==37)

76298

In [13]:
np.unique(df[df['assembly']==37]["name_db"])

array(['Clone_based_ensembl_gene', 'Clone_based_vega_gene', 'Havana gene'],
      dtype=object)

In [12]:
df[df['graph_id']=='ENSG00000173727.7']

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity,assembly
202611,107,ENSG00000173727.7,AP000769.1,Clone_based_ensembl_gene,NaN,NaN,37


In [110]:
the_dict[db_name]["Assembly"]

{'37': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False},
 '38': {'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
  'Include': False}}

In [3]:

self = ExternalDatabases(dm)
give_type='assembly'
the_dict_loaded = self.load_modified_yaml()
the_dict = the_dict_loaded[self.db_manager.organism][self.db_manager.form]

result = set()
for db_name in the_dict:
    for asm in the_dict[db_name]["Assembly"]:
        item = the_dict[db_name]["Assembly"][asm]
        res_ens = map(int, item["Ensembl release"].split(","))
        if self.db_manager.ensembl_release in res_ens and item["Include"]:
            if give_type == "db" and int(asm) == self.db_manager.ensembl_mysql_server:
                result.add(db_name)
            elif give_type == "assembly":
                result.add(int(asm))
            else:
                raise ValueError
            
            
result

{37, 38}

In [7]:
item

{'Ensembl release': '79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107',
 'Include': False}

In [6]:

for i in dm.available_releases:
    ex = ExternalDatabases(dm.change_release(i))
    ass = ex.give_list_for_case(give_type="db")
    print(f"Rel:{i}, Ass{ass}")

Rel:79, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:80, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:81, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:82, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:83, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:84, Ass['EntrezGene', 'Vega gene', 'Clone_based_ensembl_gene', 'UniProtKB Gene Name', 'Vega_gene', 'HGNC Symbol', 'Havana gene', 'Clone_based_vega_gene']
Rel:85, Ass['EntrezGene', 'Vega gene', 'Clone_based_

In [68]:
kalds=[i for i in jkl if len(jkl[i]) == 0]
len(kalds)

113

In [37]:
bok = rc37[rc37["id_db"].isin(a)]

In [35]:
dsd = []
for i in a:
    c =set([j for j in np.unique(bok[bok["id_db"] == i]["name_db"])])
    #print(c)
    dsd.extend(c)
from collections import Counter
Counter(dsd).most_common()

[('Clone-based (Vega) gene', 377),
 ('Clone_based_vega_gene', 377),
 ('Clone-based (Vega)', 377),
 ('Clone-based (Ensembl) gene', 87),
 ('Clone-based (Ensembl)', 87),
 ('Clone_based_ensembl_gene', 87),
 ('Uniprot_gn', 42),
 ('UniProtKB Gene Name', 42),
 ('EntrezGene', 41),
 ('NCBI gene (formerly Entrezgene)', 41),
 ('HGNC Symbol', 20),
 ('WikiGene', 17),
 ('RFAM', 2)]

In [38]:
boksuz

,release,graph_id,id_db,name_db,ensembl_identity,xref_identity
0,107,ENSG00000000003.10,ENSG00000000003,ArrayExpress,NaN,NaN
1,107,ENSG00000000005.5,ENSG00000000005,ArrayExpress,NaN,NaN
2,107,ENSG00000000419.8,ENSG00000000419,ArrayExpress,NaN,NaN
3,107,ENSG00000000457.9,ENSG00000000457,ArrayExpress,NaN,NaN
4,107,ENSG00000000460.12,ENSG00000000460,ArrayExpress,NaN,NaN
...,...,...,...,...,...,...
21725980,79,LRG_242.1,hsa-mir-4720,miRBase,NaN,NaN
21725981,79,LRG_368.1,MI0022645,miRBase,NaN,NaN
21725982,79,LRG_368.1,hsa-mir-6800,miRBase,NaN,NaN
21725983,79,LRG_55.1,MI0019134,miRBase,NaN,NaN


In [28]:
a

['RP11-181G12.5',
 'AL035406.1',
 'PRAMEF3',
 'C1orf134',
 'RP5-930J4.4',
 'RP1-74M1.3',
 'AL353354.1',
 'AL353354.2',
 'RP11-442N24--B.1',
 'RP1-212P9.2',
 'RP11-242O24.5',
 'FLJ00388',
 'AL645730.2',
 'RP5-850O15.4',
 'RP5-1024G6.8',
 'RP4-758J24.5',
 'AL161915.1',
 'RP11-115A15.1',
 'RP11-347K2.1',
 'RP11-347K2.2',
 'RP11-439A17.7',
 'RP11-343N15.5',
 'AL592494.5',
 'RP11-417J8.1',
 'RP11-417J8.2',
 'RP11-417J8.3',
 'RP11-417J8.6',
 'RP11-423O2.5',
 'RP11-423O2.7',
 'RP11-782C8.2',
 'RP11-782C8.1',
 'RP11-782C8.4',
 'RP11-782C8.3',
 'RP11-782C8.5',
 'RP11-435B5.3',
 'RP11-435B5.4',
 'RP11-435B5.5',
 'RP11-435B5.6',
 'RP11-435B5.7',
 'RP6-206I17.1',
 'RP6-206I17.2',
 'RP6-206I17.3',
 'AL592284.1',
 'AL590452.1',
 'RP11-458D21.6',
 'RP6-74O6.3',
 'BX842679.1',
 'XXyac-YX155B6.6',
 'RP11-666A1.5',
 'RP11-277L2.4',
 'RP11-277L2.3',
 'RP11-277L2.5',
 'RP11-353N4.1',
 'RP11-353N4.4',
 'RP11-196G18.21',
 'RP1-91G5.3',
 'EFNA3.1',
 'DCAF8.1',
 'RP11-122G18.5',
 'RP11-122G18.8',
 'RP4-782G3.